In [1]:
import numpy as np
import pandas as pd
import heapq, sys, copy
from collections import defaultdict

# Binary Bridge: Formulation and deterministic solution for shortest path

In [2]:
class BinaryBridge:
    
    def __init__(self, n_stages, mu0, sig02):
        """
        아래와 같이 생긴 이진 그래프에서 최단 거리 탐색 문제 해결(예시: n_stages=4):
                                    o
                                  /   \
                                o - o - o
                              /   /   /   \
                        시작 o - o - o - o - o 끝
               stage number 0   1   2   3   4

         - 시작부터 끝까지 가는 최단경로 탐색
         - 각 edge의 길이를 평균과 분산이 각각 mu0, sig02인 로그정규분포에서 생성
        """
        assert (n_stages % 2 == 0), 'number of stages has to be even'
        self.n_stages = n_stages
        self.mu0 = mu0
        self.sig02 = sig02
        
        self.nodes = set() # graph의 key들(node들)을 생성
        self.graph = defaultdict(dict) # 각 node에 연결된 node의 edge에 해당하는 theta를 생성
        self.minimum_distance = None
        self.shortest_path = None
        
        self._create_graph()
        self._prev = defaultdict(list)
        self._apply_dijkstra()
        
    def _get_stage_width(self, stage):
        """
        stage별 노드 개수를 반환하는 method(ex. 0->1, 1->2, 2->3, 3->2, 4->1)
        """
        width = stage - 2 * max(stage - self.n_stages/2, 0) + 1
        return int(width)
    
    def _generate_nodes(self):
        """
        노드들을 순서쌍 형태로 생성
        """
        for x in range(self.n_stages + 1):
            for y in range(self._get_stage_width(x)):
                self.nodes.add((x, y))
        
    def _generate_edges(self):
        """
        로그정규분포에서 노드들 사이의 edge에 해당하는 theta를 생성
        """
        for x in range(self.n_stages + 1):
            for y in range(self._get_stage_width(x)):
                current = (x, y)
                up_shape = (x+1, y+1)
                flat_shape = (x+1, y)
                down_shape = (x+1, y-1)
                
                if up_shape in self.nodes:
                    self.graph[current][up_shape] = np.exp(np.random.normal(self.mu0, np.sqrt(self.sig02)))
                if flat_shape in self.nodes:
                    self.graph[current][flat_shape] = np.exp(np.random.normal(self.mu0, np.sqrt(self.sig02)))
                if down_shape in self.nodes and flat_shape not in self.nodes:
                    self.graph[current][down_shape] = np.exp(np.random.normal(self.mu0, np.sqrt(self.sig02)))

                
    def _create_graph(self):
        """
        노드와 엣지를 생성해 그래프를 정의함
        """
        self._generate_nodes()
        self._generate_edges()
        
    def _apply_dijkstra(self):
        """
        Dijkstra 알고리즘을 사용해 현재 생성된 그래프 내의 최단 거리를 계산하고 최단 경로를 저장
        """
        distances = dict([(node, np.inf) for node in self.nodes])
        distances[(0,0)] = 0
        prev_path = defaultdict(list)

        to_visit = [[0, (0,0)]]
        while to_visit:
            distance, visiting = heapq.heappop(to_visit)
            for destination in self.graph[visiting].keys():
                new_distance = distance + self.graph[visiting][destination]
                if new_distance < distances[destination]:
                    distances[destination] = new_distance
                    prev_path[destination] = prev_path[visiting] + [visiting]
                    heapq.heappush(to_visit, [new_distance, destination])
        
        arrival = (self.n_stages, 0)
        self.minimum_distance = distances[arrival]
        self.shortest_path = prev_path[arrival] + [arrival]

# Environment

In [3]:
class Environment(BinaryBridge):
    
    def overwrite_edge_length(self, edge_length):
        for start_node in edge_length:
            for end_node in edge_length[start_node]:
                self.graph[start_node][end_node] = edge_length[start_node][end_node]
                
    def get_shortest_path(self):
        self._apply_dijkstra()
        return self.shortest_path
    
    def generate_outcome(self, action, sig2_tilde):
        """
        action  : list of nodes
        outcome : dictionary of elapsed time({start : {end : generated_elapse_time}})
        """
        elapsed_times = defaultdict(dict)
        for visiting, destination in zip(action, action[1:]):
            theta = self.graph[visiting][destination]
            parameter_ln = np.log(theta) - sig2_tilde / 2
            elapsed_time = np.exp(np.random.normal(parameter_ln, sig2_tilde))
            elapsed_times[visiting][destination] = elapsed_time

        return elapsed_times

# Agent

In [4]:
class EpsilonGreedy:
    
    def __init__(self, n_stages, mu0, sig02, sig2_tilde=1., epsilon=0.0):
        assert (n_stages % 2 == 0), 'number of stages has to be even'
        self.n_stages = n_stages
        self.mu0 = mu0
        self.sig02 = sig02
        self.sig2_tilde = sig2_tilde
        self.epsilon = epsilon
        
        self.internal_env = Environment(n_stages, mu0, sig02)
        self.posterior_params = copy.deepcopy(self.internal_env.graph)
        for visiting in self.posterior_params:
            for destination in self.posterior_params[visiting]:
                self.posterior_params[visiting][destination] = (mu0, sig02)
    
    def update_parameters(self, action, reward):
        
        for visiting in reward:
            for destination in reward[visiting]:
                elapse_time = reward[visiting][destination]
                previous_mean, previous_var = self.posterior_params[visiting][destination]
                
                precision_theta = 1. / previous_var
                precision_noise = 1. / self.sig2_tilde
                updated_var = 1. / (precision_theta + precision_noise)
                
                updated_mean = precision_theta * previous_mean + precision_noise * (np.log(elapse_time) + self.sig2_tilde / 2)
                updated_mean = updated_mean / (precision_theta + precision_noise)
                
                self.posterior_params[visiting][destination] = (updated_mean, updated_var)
        
    def _posterior_mean(self):
        posterior_means = copy.deepcopy(self.posterior_params)
        for visiting in self.posterior_params:
            for destination in self.posterior_params[visiting]:
                mean, var = self.posterior_params[visiting][destination]
                posterior_means[visiting][destination] = np.exp(mean + var / 2)
        
        return posterior_means
    
    def _explore(self):
        path = []
        start_node = (0,0)
        while True:
            path += [start_node]
            if start_node == (self.n_stages, 0):
                break
            start_node = random.choice(list(self.internal_env.graph.keys()))
        return path
    
    def _exploit(self):
        posterior_means = self._posterior_mean()
        self.internal_env.overwrite_edge_length(posterior_means)
        return self.internal_env.get_shortest_path()
    
    def pick_action(self):
        if np.random.rand() < self.epsilon:
            path = self._explore()
        else:
            path = self._exploit()
        return path

In [5]:
class Thompson(EpsilonGreedy):
    
    def _posterior_sample(self):
        posterior_samples = copy.deepcopy(self.posterior_params)
        for visiting in self.posterior_params:
            for destination in self.posterior_params[visiting]:
                mean, var = self.posterior_params[visiting][destination]
                posterior_samples[visiting][destination] = np.exp(np.random.normal(mean, var))
        return posterior_samples
        
    def pick_action(self):
        posterior_samples = self._posterior_sample()
        self.internal_env.overwrite_edge_length(posterior_samples)
        return self.internal_env.get_shortest_path()

# Experiment

In [6]:
def reward_function(response, aggregate=False):
    if aggregate:
        reward = 0
        for visiting in response:
            for destination in response[visiting]:
                reward += response[visiting][destination]
    else:
        reward = response
    return reward

In [7]:
class Experiment:

    def __init__(self, agent, environment, n_steps, exp_id):
        """
        agent       : predefined instance of Greedy or Thompson class
        environment : predefined instance of Environment class
        n_steps     : number of steps in current experiment
        exp_id      : id of current experiment
        """
        self.agent = agent
        self.environment = environment
        self.optimal_reward = -environment.minimum_distance
        self.n_steps = n_steps
        self.exp_id = exp_id
        self.result = []
        self.data_dict = {}

    def _step(self, step_index):
        # pick action -> generate outcome -> observe reward of the action -> update belief accordingly
        action = self.agent.pick_action()
        response = self.environment.generate_outcome(action, self.agent.sig2_tilde)
        self.agent.update_parameters(action, reward_function(response))

        # calculate regret of current action
        expected_reward = 0
        for visiting, destination in zip(action, action[1:]):
            expected_reward -= self.environment.graph[visiting][destination]
        regret = self.optimal_reward - expected_reward

        # Leave log
        self.cum_regret += regret
        self.data_dict = {'step': (step_index + 1), 
                          'regret': regret, 
                          'total_dist': reward_function(response, True), 
                          'action': action, 
                          'experiment_id':self.exp_id}
        self.result.append(self.data_dict)
    
    def run(self):
        self.cum_regret = 0
        for t in range(self.n_steps):
            self._step(t)
        self.result = pd.DataFrame(self.result)

In [8]:
results = []
n_stages = 20
mu0 = -0.5
sig02 = 1

n_steps = 500
n_experiment = 100
environment = Environment(n_stages, mu0, sig02)

for exp_id in range(1, n_experiment+1):
    sys.stdout.write('\r')
    sys.stdout.write(str(exp_id))
    sys.stdout.flush()
    
    np.random.seed(exp_id)
    for agent_type in ['greedy', 'ts']:
        agent = EpsilonGreedy(n_stages, mu0, sig02) if agent_type == 'greedy' else Thompson(n_stages, mu0, sig02)
        experiment = Experiment(agent, environment, n_steps, exp_id)
        experiment.run()
        experiment.result.insert(experiment.result.shape[1], 'agent', agent_type)
        results.append(experiment.result)

100